In [1]:
# Check GPU
!nvidia-smi


Sat Oct 25 13:33:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# Install required libraries
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate safetensors pillow


In [4]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import os

# Select model (you can change to any HF model like "stabilityai/stable-diffusion-2-1")
MODEL_NAME = "runwayml/stable-diffusion-v1-5"

# Detect GPU/CPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Create output folder
os.makedirs("outputs", exist_ok=True)

# Load Stable Diffusion pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
)
pipe = pipe.to(DEVICE)


Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


In [5]:
prompt = "A serene landscape of mountains reflected in a crystal-clear lake, cinematic lighting, ultra-realistic, 8K resolution"

# Optional: add negative prompt
negative_prompt = "low quality, blurry, watermark, text, distorted"

# For reproducibility
seed = 123
generator = torch.Generator(device=DEVICE).manual_seed(seed)

# Generate
image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    guidance_scale=7.5,
    num_inference_steps=30,
    generator=generator
).images[0]

# Save and show
output_path = f"outputs/generated_image.png"
image.save(output_path)
image.show()
print("✅ Image saved to:", output_path)


  0%|          | 0/30 [00:00<?, ?it/s]

✅ Image saved to: outputs/generated_image.png


In [6]:
prompts = [
    "A futuristic city skyline at sunset, neon reflections, ultra-detailed",
    "A cozy coffee shop interior with vintage decor, warm lighting, 4K",
    "An astronaut riding a horse on Mars, cinematic composition"
]

for i, p in enumerate(prompts):
    generator = torch.Generator(device=DEVICE).manual_seed(100 + i)
    img = pipe(p, guidance_scale=7.5, num_inference_steps=30, generator=generator).images[0]
    img.save(f"outputs/prompt_{i+1}.png")
    print(f"Saved outputs/prompt_{i+1}.png")


  0%|          | 0/30 [00:00<?, ?it/s]

Saved outputs/prompt_1.png


  0%|          | 0/30 [00:00<?, ?it/s]

Saved outputs/prompt_2.png


  0%|          | 0/30 [00:00<?, ?it/s]

Saved outputs/prompt_3.png
